# ETL 

Lo primero que hacemos es llamar las funciones: `Fitizens_libraries` es la carpeta en la que se encuentan los archivos .py con las funciones:

In [ ]:
from fitizens_libraries.load_and_process_training_data import load_training_data
from fitizens_libraries.load_timeseries import load_timeseries_data
from custom_libraries.merge_data import merge_data
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from fitizens_libraries.plot_labeled_sequences import plot_labeled_sequence
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from fitizens_libraries.build_dataframe_from_list_of_signals import build_dataframe
import pandas as pd
import os

Para el proceso de ETL lo que haremos será utilizar la función de `load_training_data`:
1. Basicamente nos busca los archivos JSON en la carpeta zip en la que se encuentren y los carga.
2. Intenta buscar repeticiones falsas y verdaderas del ejercicio 
3. Se filtran las repeticiones falsas para que tengan una duración dentro del rango de las repeticiones reales.

Es importante tener en cuenta que la función me exige unos hiperparametros obligatorios que tengo que indicar:

- signals : list of str
    List of signals to include in the dataframe. The signals must be present in the input data
- target_exercise : str
        Exercise to detect repetitions
- other_exercises : list of str
        List of exercises to use as negative examples

In [ ]:
folder_path = "LABELED"
os.makedirs(folder_path, exist_ok=True) #Referenciamos la carpeta LABELED en la que están las carpetas zip con los json
#Ahora voy a iterar en esa carpeta LABELED para obtener la ruta de los archivos, que es el LABELED/NOMBRE y eso lo guardo en una lista
file_names = []
for name in os.listdir(folder_path):
    file_path = f"{folder_path}/{name}"
    file_names.append(file_path)
#Ahora tengo que especificar mis features 
signals = ["accX", "accY", "accZ", "gyroX", "gyroY", "gyroZ", "magnX", "magnY", "magnZ", "linAccX", "linAccY", "linAccZ"]
#Indico mi target
target_exercise="SQUAT"

In [ ]:
len(file_names)

In [ ]:
data,wk = load_training_data(filelist=file_names,
                         signals= signals,
                          target_exercise=target_exercise, other_exercises=[], is_peak_minima=True)

In [ ]:
#Revisamos
#data
len(data)

In [ ]:
df = merge_data(data)
len(df)

In [ ]:
df.head()

In [ ]:
df.columns

Ahora voy a convertir esto en un problema de clasificacion binario; para ello, voy a crear una columna de exercise, en la que segun la columna `exercising_periods` me diga si hay o no un squad. Si esa columna tiene un cero, indica que no hay squad, de lo contrario es un squad.

In [ ]:
def nueva_columna(exercise):
    if exercise == 0:
        return 'no exercise'
    else:
        return 'squad'
df['squad'] = df['exercising_periods'].apply(nueva_columna)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#revisar porcentaje de valores nulos por columnas
((df.isnull().sum())/len(df))*100

In [ ]:
sns.countplot(x=df['squad'], label = "squad")

In [ ]:
df.index

In [ ]:
fig = px.line(df, x=df.index, y='linAccZ', title='Time serie of exercise linAccZ',color='squad')
fig.show()

In [ ]:
fig = px.box(df, y="linAccZ", color="squad",title = 'Distribution of linAccZ vs target variable')
fig.update_traces(quartilemethod="exclusive") 
fig.show()

In [ ]:
fig = px.box(df, y="linAccY", color="squad",title = 'Distribution of linAccY vs target variable')
fig.update_traces(quartilemethod="exclusive") 
fig.show()

In [ ]:
fig = px.box(df, y="linAccX", color="squad",title = 'Distribution of linAccX vs target variable')
fig.update_traces(quartilemethod="exclusive") 
fig.show()

In [ ]:
fig = px.box(df, y="accZ_mod", color="squad",title = 'Distribution of accZ_mod vs target variable')
fig.update_traces(quartilemethod="exclusive") 
fig.show()

In [ ]:
fig = px.box(df, y="accX_mod", color="squad",title = 'Distribution of accX_mod vs target variable')
fig.update_traces(quartilemethod="exclusive") 
fig.show()

In [ ]:
fig = px.box(df, y="accY_mod", color="squad",title = 'Distribution of accY_mod vs target variable')
fig.update_traces(quartilemethod="exclusive") 
fig.show()

In [ ]:
timeseries, labels_ranges = load_timeseries_data(filelist=file_names, signals=signals, is_peak_minima=True)

In [ ]:
timeseries.columns

In [ ]:
plot_labeled_sequence(timeseries, labels_ranges[0:5])

In [ ]:
labels_ranges

In [ ]:
y = df['exercising_periods']

num_lags=24 # to discuss

plt.plot(y)
plt.show()
plot_acf(y,lags=num_lags)
plt.show()
plot_pacf(y,lags=num_lags,method="ols")
plt.show()